## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2
import yaml

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Hyperparameter Setting

In [3]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing
    한글 폴더 변경필요

In [5]:
FolderPath = r'C:\Users\wjswp\Dacon\papering_data\train'
# os.chdir(FolderPath)
# for idx,i in enumerate(os.listdir(FolderPath)):
#     os.rename(i,i.replace(i,str(idx)))


In [6]:
all_img_list = glob.glob(FolderPath+'/*/*')
all_img_list[:5]

['C:\\Users\\wjswp\\Dacon\\papering_data\\train\\0\\0.png',
 'C:\\Users\\wjswp\\Dacon\\papering_data\\train\\0\\1.png',
 'C:\\Users\\wjswp\\Dacon\\papering_data\\train\\0\\10.png',
 'C:\\Users\\wjswp\\Dacon\\papering_data\\train\\0\\11.png',
 'C:\\Users\\wjswp\\Dacon\\papering_data\\train\\0\\2.png']

In [7]:
all_img_list[0].split('\\')[-2]

'0'

In [8]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('\\')[-2])

In [9]:
df.tail()

,img_path,label
3452,C:\Users\wjswp\Dacon\papering_data\train\9\56.png,9
3453,C:\Users\wjswp\Dacon\papering_data\train\9\6.png,9
3454,C:\Users\wjswp\Dacon\papering_data\train\9\7.png,9
3455,C:\Users\wjswp\Dacon\papering_data\train\9\8.png,9
3456,C:\Users\wjswp\Dacon\papering_data\train\9\9.png,9


In [10]:
# dic = {}
# for idx,i in enumerate(list(df.label.unique())):
#     dic[idx] = i
# dic

# with open('labelChanger.yaml','w',encoding='utf-8') as f:
#     yaml.dump(dic,f,allow_unicode=True)
#     f.close()

In [11]:
with open('labelChanger.yaml','r',encoding='utf-8') as f:
    text = yaml.load(f,Loader=yaml.FullLoader)
    # print(text)
text.items()

dict_items([(0, '가구수정'), (1, '걸레받이수정'), (2, '곰팡이'), (3, '꼬임'), (4, '녹오염'), (5, '들뜸'), (6, '면불량'), (7, '몰딩수정'), (8, '반점'), (9, '석고수정'), (10, '오염'), (11, '오타공'), (12, '울음'), (13, '이음부불량'), (14, '창틀,문틀수정'), (15, '터짐'), (16, '틈새과다'), (17, '피스'), (18, '훼손')])

In [12]:
# reverse_text = {v:k for k,v in text.items()}
# df['label_index'] = df.apply(lambda x: reverse_text[x['label']],axis=1)
# df.drop(columns='label')

In [13]:
im = r'C:\Users\wjswp\Dacon\papering_data\train\0\0.png'
ex = cv2.imread(rf'{im}')
ex

array([[[115, 114, 110],
        [116, 115, 111],
        [114, 113, 109],
        ...,
        [128, 128, 134],
        [129, 129, 135],
        [125, 125, 131]],

       [[114, 113, 109],
        [114, 113, 109],
        [117, 116, 112],
        ...,
        [130, 130, 136],
        [130, 130, 136],
        [128, 128, 134]],

       [[116, 115, 111],
        [117, 116, 112],
        [116, 115, 111],
        ...,
        [130, 130, 136],
        [128, 128, 134],
        [127, 127, 133]],

       ...,

       [[110, 108, 107],
        [110, 108, 107],
        [111, 109, 108],
        ...,
        [130, 128, 136],
        [131, 128, 137],
        [134, 131, 140]],

       [[112, 110, 109],
        [110, 108, 107],
        [109, 107, 106],
        ...,
        [129, 128, 136],
        [128, 127, 136],
        [129, 128, 136]],

       [[112, 110, 109],
        [110, 108, 107],
        [110, 108, 107],
        ...,
        [128, 128, 134],
        [129, 129, 135],
        [127, 127, 133]]

In [14]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

## Label-Encoding

In [15]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

## CustomDataset

In [16]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [17]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [18]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [19]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

## Train

In [20]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [21]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.type(torch.LongTensor)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

## Run!!

In [22]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.39544] Val Loss : [0.91340] Val Weighted F1 Score : [0.66922]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

...

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.04885] Val Loss : [1.10074] Val Weighted F1 Score : [0.75368]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.09008] Val Loss : [1.21649] Val Weighted F1 Score : [0.75639]


## Inference

In [23]:
infPath = r'C:\Users\wjswp\Dacon\papering_data/'
test = pd.read_csv(infPath+'test.csv')

In [24]:
test

,id,img_path
0,TEST_000,./test/000.png
1,TEST_001,./test/001.png
2,TEST_002,./test/002.png
3,TEST_003,./test/003.png
4,TEST_004,./test/004.png
...,...,...
787,TEST_787,./test/787.png
788,TEST_788,./test/788.png
789,TEST_789,./test/789.png
790,TEST_790,./test/790.png


In [25]:
os.chdir(infPath)
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [26]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [27]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

In [28]:
os.chdir(r'C:\Users\wjswp\participate_competition\Dacon\papering_detect')
with open('labelChanger.yaml','r',encoding='utf-8') as f:
    text = yaml.load(f,Loader=yaml.FullLoader)
    # print(text)
text.items()

dict_items([(0, '가구수정'), (1, '걸레받이수정'), (2, '곰팡이'), (3, '꼬임'), (4, '녹오염'), (5, '들뜸'), (6, '면불량'), (7, '몰딩수정'), (8, '반점'), (9, '석고수정'), (10, '오염'), (11, '오타공'), (12, '울음'), (13, '이음부불량'), (14, '창틀,문틀수정'), (15, '터짐'), (16, '틈새과다'), (17, '피스'), (18, '훼손')])

## Submission

In [29]:
submit = pd.read_csv(infPath+'sample_submission.csv')

In [30]:
submit['label'] = preds

In [31]:
submit

,id,label
0,TEST_000,18
1,TEST_001,10
2,TEST_002,18
3,TEST_003,7
4,TEST_004,10
...,...,...
787,TEST_787,18
788,TEST_788,18
789,TEST_789,10
790,TEST_790,10


In [32]:
submit['label'] = submit.apply(lambda x: text[int(x['label'])],axis=1)
submit

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,훼손
788,TEST_788,훼손
789,TEST_789,오염
790,TEST_790,오염


In [33]:
submit.to_csv(infPath+'baseline_submit.csv', index=False,encoding='utf-8-sig')